## 导入必要的库

In [1]:
import torch

## 基本示例：理解 Node 和 Edge

创建叶子节点并构建简单的计算图

In [2]:
# 1. 创建叶子节点 (AccumulateGrad 节点的前身)
x = torch.tensor([2.0], requires_grad=True)
y = torch.tensor([3.0], requires_grad=True)

# 2. 执行前向计算，构建图
# z = x * y  -> 产生 MulBackward0 Node
# out = z + 1 -> 产生 AddBackward0 Node
z = x * y
out = z + 1

## 探索计算图结构

In [3]:
print(f"Output Node (grad_fn): {out.grad_fn}")
print(f"Input Edges for Output: {out.grad_fn.next_functions}")

Output Node (grad_fn): <AddBackward0 object at 0x111d20880>
Input Edges for Output: ((<MulBackward0 object at 0x10a32a710>, 0), (None, 0))


In [4]:
# 深入一层，查看 z 的 Node
z_node = out.grad_fn.next_functions[0][0]
print(f"\nPrevious Node (z): {z_node}")
print(f"Input Edges for z: {z_node.next_functions}")


Previous Node (z): <MulBackward0 object at 0x10a3337f0>
Input Edges for z: ((<AccumulateGrad object at 0x10a2a02e0>, 0), (<AccumulateGrad object at 0x10a2a1b10>, 0))


In [5]:
# 继续深入，查看 x 和 y 的 Node
x_node = z_node.next_functions[0][0]
y_node = z_node.next_functions[1][0]
print(f"Input Edges for x: {x_node.next_functions}")
print(f"Input Edges for y: {y_node.next_functions}")
print(f"Leaf Node (x): {x_node}")
print(f"Leaf Node (y): {y_node}")

Input Edges for x: ()
Input Edges for y: ()
Leaf Node (x): <AccumulateGrad object at 0x10a2a1d80>
Leaf Node (y): <AccumulateGrad object at 0x10a2a1db0>


### 解释 Node 和 Edge 之间的关系

- **Node**：表示一个具体的操作（如加法、乘法等），对应于计算图中的一个结点。
- **Edge**：表示节点之间的数据流动关系，即一个节点的输出作为另一个节点的输入。

在上面的例子中：
- `out` 节点是一个 AddBackward 节点，表示执行了加法操作。
- `out` 节点有两个输入边（Edges），分别指向 z 节点和一个常数 1（常数没有对应的 Node，因此是 None）。
- `z` 节点是一个 MulBackward 节点，表示执行了乘法操作。
- `z` 节点有两个输入边，分别指向 x 和 y 节点，这两个节点都是叶子节点（AccumulateGrad 节点）。

## 复杂计算图示例

构建一个更复杂的计算图来展示多层结构

In [6]:
# 新增叶子结点 a b c
a = torch.tensor([4.0], requires_grad=True)
b = torch.tensor([5.0], requires_grad=True)
c = torch.tensor([6.0], requires_grad=True)

# 构建更复杂的计算图
d = (a + b + c) / (x * y)
print(f"\nComplex Output Node (d) grad_fn: {d.grad_fn}")
print(f"Input Edges for d: {d.grad_fn.next_functions}")


Complex Output Node (d) grad_fn: <DivBackward0 object at 0x113a27520>
Input Edges for d: ((<AddBackward0 object at 0x10a416f80>, 0), (<MulBackward0 object at 0x10a43c190>, 0))


In [7]:
# 查看 d 的计算图的更深层次结构
add_node = d.grad_fn.next_functions[0][0]
mul_node = d.grad_fn.next_functions[1][0]
print(f"\nAddition Node in d: {add_node}")
print(f"Input Edges for Addition Node: {add_node.next_functions}")
add_node_left = add_node.next_functions[0][0]
add_node_right = add_node.next_functions[1][0]
print(f"Left Input Edges for Addition Node: {add_node_left.next_functions}")
print(f"Right Input Edges for Addition Node: {add_node_right.next_functions}")

print(f"\nMultiplication Node in d: {mul_node}")
print(f"Input Edges for Multiplication Node: {mul_node.next_functions}")


Addition Node in d: <AddBackward0 object at 0x10a3303a0>
Input Edges for Addition Node: ((<AddBackward0 object at 0x10a332260>, 0), (<AccumulateGrad object at 0x10a333370>, 0))
Left Input Edges for Addition Node: ((<AccumulateGrad object at 0x10a2a2ef0>, 0), (<AccumulateGrad object at 0x10a2a3400>, 0))
Right Input Edges for Addition Node: ()

Multiplication Node in d: <MulBackward0 object at 0x10a333940>
Input Edges for Multiplication Node: ((<AccumulateGrad object at 0x10a2a1d80>, 0), (<AccumulateGrad object at 0x10a2a1db0>, 0))


## AccumulateGrad：叶子节点的梯度累加器

AccumulateGrad 是叶子节点（变量）的梯度收集节点。

### 为什么叫 "Accumulate" (累加)？

在反向传播结束时，计算出的梯度需要存入变量的 `.grad` 属性中。如果一个变量在计算图中被使用了多次（例如 `y = x * x`），会有多条路径传回梯度。PyTorch 的机制是将这些梯度累加（Accumulate）到 `.grad` 中，而不是覆盖它。因此，这个节点的名字叫 AccumulateGrad，代表"将梯度累加到叶子节点的 .grad 属性里"。

## 梯度累加示例

In [8]:
# 模拟一个共享权重 w
w = torch.tensor([1.0], requires_grad=True)

# 路径 1: Loss1 = w * 2 -> d(Loss1)/dw = 2
loss1 = w * 2
loss1.backward()
print(f"Step 1 - w.grad (Expected 2.0): {w.grad.item()}")

Step 1 - w.grad (Expected 2.0): 2.0


In [9]:
# 路径 2: Loss2 = w * 3 -> d(Loss2)/dw = 3
# 注意：我们需要再次构建计算图，因为上一次已经被销毁
loss2 = w * 3
loss2.backward()

# 关键点：此时 w.grad 不是 3.0，而是 2.0 + 3.0 = 5.0
print(f"Step 2 - w.grad (Expected 2.0 + 3.0 = 5.0): {w.grad.item()}")

Step 2 - w.grad (Expected 2.0 + 3.0 = 5.0): 5.0


In [10]:
# 手动清零 (模拟 optimizer.zero_grad())
w.grad.zero_()
print(f"Step 3 - After zero_grad: {w.grad.item()}")

Step 3 - After zero_grad: 0.0


### 重要结论

当 `backward()` 触及叶子节点 w 时，它触发了 w 关联的 AccumulateGrad Node。这个 Node 执行的操作实质上是 `w.grad = w.grad + new_grad`。

这就是为什么在训练循环中，必须在 `loss.backward()` 之前调用 `optimizer.zero_grad()`，否则梯度会跨越 batch 累积，导致参数更新方向错误。